In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# get cleaned data
cleaned_data = pd.read_csv('src/cleaned_data.csv')

In [14]:
cleaned_data.isnull().sum().sum()

0

In [15]:
# check if there is any inf or -inf value
np.isinf(cleaned_data).sum().sum()

0

In [16]:
cleaned_data['BUREAU_SK_ID_BUREAU_COUNT'].value_counts()

0.0      44020
1.0      36072
2.0      35635
3.0      32925
4.0      28973
         ...  
61.0         1
58.0         1
53.0         1
116.0        1
59.0         1
Name: BUREAU_SK_ID_BUREAU_COUNT, Length: 61, dtype: int64

In [28]:
possible_feature = pd.DataFrame()
possible_feature['SK_ID_CURR'] = cleaned_data['SK_ID_CURR']
#Average frequency of taking out loans
possible_feature['Freq_take_loan_avg'] = (cleaned_data['BUREAU_DAYS_CREDIT_max'] - cleaned_data[
    'BUREAU_DAYS_CREDIT_min']) / 30 / cleaned_data.BUREAU_SK_ID_BUREAU_COUNT
#Percentage of previous instalment paid 
possible_feature['Pct_prev_ins_paid'] = (cleaned_data.INS_AMT_PAYMENT_sum / cleaned_data.INS_AMT_INSTALMENT_sum) ** 4
#Average loan amount approved previously for each month
possible_feature[
    'avg_credit_approval_per_month'] = cleaned_data.PREV_APP_AMT_CREDIT_sum / cleaned_data.PREV_APP_CNT_PAYMENT_sum
#Previous annuity amount as pct of income
possible_feature['Pct_prev_annuity_in_income'] = (cleaned_data.PREV_APP_AMT_ANNUITY_mean * np.exp(
    0.05 * abs(cleaned_data.PREV_APP_DAYS_DECISION_mean) / 365)) / cleaned_data.APP_AMT_INCOME_TOTAL
#Work percentage of life
possible_feature['Pct_work_life'] = cleaned_data.APP_DAYS_EMPLOYED / cleaned_data.APP_DAYS_BIRTH
#good price as pct of credit amount
possible_feature[
    'Pct_good_p_in_credit_prev'] = cleaned_data.PREV_APP_AMT_GOODS_PRICE_sum / cleaned_data.PREV_APP_AMT_CREDIT_sum
possible_feature['Pct_good_p_in_credit_curr'] = cleaned_data.APP_AMT_GOODS_PRICE / cleaned_data.APP_AMT_CREDIT
#amount of instal overdue per day
possible_feature['Amt_ins_overdue_daily'] = cleaned_data.INS_AMT_INSTALMENT_sum / abs(
    cleaned_data.INS_DAYS_ENTRY_PAYMENT_sum - cleaned_data.INS_DAYS_INSTALMENT_sum)
# prob runaway
possible_feature['runaway'] = np.sqrt(
    cleaned_data.APP_DAYS_ID_PUBLISH * cleaned_data.APP_DAYS_LAST_PHONE_CHANGE) * cleaned_data.CNT_DRAWINGS_CURRENT * cleaned_data.CC_AMT_DRAWINGS_CURRENT_sum
possible_feature['approval_imbalance'] = (
                                                 cleaned_data.PREV_APP_AMT_APPLICATION_sum - cleaned_data.PREV_APP_AMT_CREDIT_sum) / (
                                                 cleaned_data.PREV_APP_AMT_APPLICATION_sum + cleaned_data.PREV_APP_AMT_CREDIT_sum)
possible_feature[
    'downpayment_percent'] = cleaned_data.PREV_APP_AMT_DOWN_PAYMENT_sum / cleaned_data.PREV_APP_AMT_GOODS_PRICE_sum
possible_feature[
    'avg_atm_withdrawl'] = cleaned_data.CC_AMT_DRAWINGS_ATM_CURRENT_sum / cleaned_data.CNT_DRAWINGS_ATM_CURRENT
possible_feature[
    'overdue_historical_comparison'] = cleaned_data.BUREAU_AMT_CREDIT_SUM_OVERDUE_max / cleaned_data.BUREAU_AMT_CREDIT_MAX_OVERDUE_max
possible_feature['industry_trend'] = np.exp(cleaned_data.APP_AMT_INCOME_TOTAL / (
        cleaned_data.APP_DEF_60_CNT_SOCIAL_CIRCLE / np.sqrt(cleaned_data.APP_REGION_POPULATION_RELATIVE)))


C:\Users\kevin\AppData\Roaming\Python\Python38\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


# NA features:

In [18]:
['Freq_take_loan_avg',
 'Pct_prev_ins_paid',
 'avg_credit_approval_per_month',
 'Pct_prev_annuity_in_income',
 'Pct_work_life', 'Pct_good_p_in_credit_prev',
 'Pct_good_p_in_credit_curr',
 'Amt_ins_overdue_daily',
 'approval_imbalance',
 'downpayment_percent',
 'avg_atm_withdrawl',
 'overdue_historical_comparison',
 'industry_trend']

['Freq_take_loan_avg',
 'Pct_prev_ins_paid',
 'avg_credit_approval_per_month',
 'Pct_prev_annuity_in_income',
 'Pct_work_life',
 'Pct_good_p_in_credit_prev',
 'Pct_good_p_in_credit_curr',
 'Amt_ins_overdue_daily',
 'approval_imbalance',
 'downpayment_percent',
 'avg_atm_withdrawl',
 'overdue_historical_comparison',
 'industry_trend']

## Freq_take_loan_avg: reason: BUREAU_SK_ID_BUREAU_COUNT can be 0. div by 0

In [19]:
cleaned_data['BUREAU_SK_ID_BUREAU_COUNT'].value_counts()

0.0      44020
1.0      36072
2.0      35635
3.0      32925
4.0      28973
         ...  
61.0         1
58.0         1
53.0         1
116.0        1
59.0         1
Name: BUREAU_SK_ID_BUREAU_COUNT, Length: 61, dtype: int64

## Pct_prev_ins_paid: div 0

In [21]:
cleaned_data['INS_AMT_INSTALMENT_sum'].value_counts()

0.000          15871
204935.580        17
56555.010         16
127421.280        12
235291.725        10
               ...  
243749.565         1
1424915.955        1
1625081.760        1
20843.685          1
3068388.810        1
Name: INS_AMT_INSTALMENT_sum, Length: 287906, dtype: int64

In [26]:
temp = cleaned_data.loc[:,['SK_ID_CURR','INS_AMT_INSTALMENT_sum']]
temp[temp['INS_AMT_INSTALMENT_sum'] == 0]

,SK_ID_CURR,INS_AMT_INSTALMENT_sum
20,100024,0.0
26,100031,0.0
50,100058,0.0
64,100076,0.0
81,100096,0.0
...,...,...
307387,456113,0.0
307422,456155,0.0
307439,456177,0.0
307449,456187,0.0


In [31]:
possible_feature[possible_feature['SK_ID_CURR']==100024]['Pct_prev_ins_paid']

20   NaN
Name: Pct_prev_ins_paid, dtype: float64

## avg_credit_approval_per_month: div 0

In [32]:
cleaned_data['PREV_APP_CNT_PAYMENT_sum'].value_counts()

12.0      29058
24.0      19015
0.0       18945
6.0       14545
10.0      13191
          ...  
1056.0        1
459.0         1
842.0         1
911.0         1
964.0         1
Name: PREV_APP_CNT_PAYMENT_sum, Length: 727, dtype: int64

## Pct_prev_annuity_in_income: div 0

In [33]:
cleaned_data['APP_AMT_INCOME_TOTAL'].value_counts()

0.0         128438
135000.0     20821
112500.0     18042
157500.0     15483
180000.0     14464
             ...  
67275.0          1
189148.5         1
100476.0         1
183825.0         1
78664.5          1
Name: APP_AMT_INCOME_TOTAL, Length: 1702, dtype: int64

## Pct_work_life: div0 from APP_DAYS_BIRTH. fixed in the new version of cleaned_data: if APP_DAYS_BIRTH == 0, then APP_DAYS_BIRTH should be nan instead of 0

In [34]:
cleaned_data['APP_DAYS_BIRTH'].value_counts()

 0.0        128438
-10292.0        30
-15771.0        28
-13481.0        28
-12988.0        27
             ...  
-7852.0          1
-24328.0         1
-25036.0         1
-24042.0         1
-7988.0          1
Name: APP_DAYS_BIRTH, Length: 17326, dtype: int64

## Pct_good_p_in_credit_curr: div 0

In [36]:
cleaned_data['APP_AMT_CREDIT'].value_counts()

0.0          128438
450000.0       5669
675000.0       5123
225000.0       4729
270000.0       4249
              ...  
1411038.0         1
1552221.0         1
1399122.0         1
357552.0          1
321970.5          1
Name: APP_AMT_CREDIT, Length: 4835, dtype: int64

## Pct_good_p_in_credit_curr: div 0

In [37]:
cleaned_data['APP_AMT_CREDIT'].value_counts()

0.0          128438
450000.0       5669
675000.0       5123
225000.0       4729
270000.0       4249
              ...  
1411038.0         1
1552221.0         1
1399122.0         1
357552.0          1
321970.5          1
Name: APP_AMT_CREDIT, Length: 4835, dtype: int64

## Amt_ins_overdue_daily: div 0. fix: replace 0 with -1 to avoid div 0 but still keep the order information, or do transformation to some distribution function to avoid div 0

In [39]:
cleaned_data['INS_DAYS_ENTRY_PAYMENT_sum'].value_counts()

 0.0         15876
-1997.0         28
-2294.0         27
-1675.0         27
-1197.0         27
             ...  
-119029.0        1
-94423.0         1
-88616.0         1
-103001.0        1
-215206.0        1
Name: INS_DAYS_ENTRY_PAYMENT_sum, Length: 102909, dtype: int64

## approval_imbalance: div 0.

In [41]:
cleaned_data['PREV_APP_AMT_APPLICATION_sum'].value_counts()

0.0          17469
225000.0      1107
135000.0      1025
45000.0        844
450000.0       767
             ...  
1361299.5        1
1333453.5        1
3411499.5        1
1136192.4        1
2902167.0        1
Name: PREV_APP_AMT_APPLICATION_sum, Length: 175559, dtype: int64

In [42]:
cleaned_data['PREV_APP_AMT_CREDIT_sum'].value_counts()

0.00          16681
45000.00        590
135000.00       422
225000.00       402
90000.00        338
              ...  
141970.50         1
1728972.18        1
918981.00         1
731011.50         1
3395448.00        1
Name: PREV_APP_AMT_CREDIT_sum, Length: 194245, dtype: int64

## downpayment_percent div 0

In [43]:
cleaned_data['PREV_APP_AMT_GOODS_PRICE_sum'].value_counts()

0.000          17469
225000.000      1106
135000.000      1025
45000.000        844
450000.000       767
               ...  
859855.950         1
329962.500         1
464557.500         1
79109.595          1
2902167.000        1
Name: PREV_APP_AMT_GOODS_PRICE_sum, Length: 175570, dtype: int64

## avg_atm_withdrawl: div 0, handle with care. smallest non-zero value is 3

In [44]:
cleaned_data['CNT_DRAWINGS_ATM_CURRENT'].value_counts()

0.0     256223
3.0       8949
4.0       8020
2.0       7797
5.0       5862
1.0       5308
6.0       4303
7.0       2931
8.0       2254
9.0       1493
10.0      1189
11.0       756
12.0       551
13.0       430
14.0       341
15.0       247
16.0       196
17.0       143
18.0       118
19.0        88
20.0        64
21.0        56
22.0        45
24.0        22
23.0        21
25.0        18
28.0        14
26.0        11
31.0        10
27.0        10
30.0         8
29.0         7
35.0         4
33.0         4
34.0         3
36.0         3
37.0         2
39.0         2
38.0         2
41.0         2
32.0         1
44.0         1
43.0         1
51.0         1
Name: CNT_DRAWINGS_ATM_CURRENT, dtype: int64

## overdue_historical_comparison: div 0: this 0 should probably be set to -1 to avoid div 0 but represent complete of payment 

In [45]:
cleaned_data['BUREAU_AMT_CREDIT_SUM_OVERDUE_max'].value_counts()

0.0        304177
4.5           265
9.0            96
13.5           66
18.0           58
            ...  
29767.5         1
28143.0         1
34389.0         1
54315.0         1
4158.0          1
Name: BUREAU_AMT_CREDIT_SUM_OVERDUE_max, Length: 1203, dtype: int64

## industry_trend: div 0 by sqrt(0). handle with care since the smallest non-zero number is 0.035

In [46]:
cleaned_data['APP_REGION_POPULATION_RELATIVE'].value_counts()

0.000000    128438
0.035792      9602
0.046220      7835
0.030755      7067
0.025164      6936
             ...  
0.001417       257
0.001333       130
0.000533        24
0.000938        14
0.000290         2
Name: APP_REGION_POPULATION_RELATIVE, Length: 82, dtype: int64

In [47]:
np.sqrt(0)

0.0